# CA 1
## Amirhossein Alizad 810197546
## Search algorithms

<strong> in this computer assignment we are going to implement and find the differences between uninformed search algorithms (BFS and IDS) and informed search algorithms (A* and weighteA*).</strong>

<strong>as it is said in the description we initially have an n*m table where a snake with size 1 is placed in one of them and there are also seeds with +1 or +2 points in some of them.<br>
the goal is to eat all the seeds with minimum cost and not to cut its self. </strong>

<strong> first we implement our classes. the class <u> State </u> holds the informations of our states that makes it different to other states</strong><br>
```python
class State:
    def __init__(self, snake, seeds):
        self.snake = snake  #it holds a Tuple of Tuples where snake[0] is tail and snake[-1] is its head
        self.seeds = seeds  #the coordinate of every seed and how many seeds we have left in every block
```
<br>
<strong> then we implement our class <u> Node </u> which holds our every state, its parent, its action, how many seeds eaten before this state, length of the snake, number of moves to reach this state, heuristic which holds the heuristic value for every state and almost which holds ( self.depth + self.heuristic ) </strong>
<br>
<strong> we have also overridden <u>hash</u> and <u>eq</u> functions for State class to know that if another state is different from another or not</strong>
<br>
<strong> we also have a class named <u> Heap </u> which we will use for our A* search to decrease the time to insert and delete from our list or queue or stack etc.</strong>

Classes
=

In [1]:
#classes
class Node:
    def __init__(self, state, eaten, depth = 0):
        self.state = state
        self.parent = None
        self.action = None
        self.eaten = eaten
        self.length = len(state.snake)
        self.depth = depth
        self.heuristic = None
        self.almost = None
        
    def set_p_a(self, parent, action):
        self.parent = parent
        self.action = action

class State:
    def __init__(self, snake, seeds):
        self.snake = snake
        self.seeds = seeds

    def __hash__(self):
        return hash((self.snake, self.seeds))

    def __eq__(self, other):
        return (self.snake == other.snake and self.seeds == other.seeds)

class Heap(object):
    def __init__(self, heap=None):
        self.heap = [] if heap is None else heap

    def _inv_heapify(self, child_index):
        heap = self.heap
        child = child_index
        while child > 0:
            parent = child // 2
            if heap[parent].almost <= heap[child].almost:
                return
            heap[parent], heap[child] = heap[child], heap[parent]
            child = parent

    def _heapify(self, parent_index):
        heap = self.heap
        length = len(heap)
        if length == 1:
            return
        parent = parent_index
        while 2 * parent < length:
            child = 2 * parent
            if child + 1 < length and heap[child + 1].almost <  heap[child].almost:
                child += 1
            if heap[parent].almost <= heap[child].almost:
                return
            heap[parent], heap[child] = heap[child], heap[parent]
            parent = child

    def del_min(self):
        heap = self.heap
        last_element = heap.pop()
        if not heap:
            return last_element
        item = heap[0]
        heap[0] = last_element
        self._heapify(0)
        return item

    def min(self):
        if not self.heap:
            return None
        return self.heap[0]

    def add(self, element):
        self.heap.append(element)
        self._inv_heapify(len(self.heap) - 1)


# BFS

In [2]:
#BFS function
def BFS(queue, explored, n, m, num_of_seeds, seeds):
    expand = 1
    all_states = 0
    while(queue):
        curr = queue.pop(0)
        expand += 1
        curr_x = curr.state.snake[-1][0]
        curr_y = curr.state.snake[-1][1]
        eaten = 0
        p = 0
        if ((curr_x, curr_y)) in seeds:
            idx = seeds.index((curr_x, curr_y))
            p = curr.state.seeds[idx][2]
            if p > 0:
                eaten = 1
                idx = curr.state.seeds.index((curr_x, curr_y, p))
                if curr.eaten + 1 == num_of_seeds:
                    return curr, expand, all_states

        if ((((curr_x - 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or (((curr_x - 1) % n , curr_y % m)) not in curr.state.snake:
            if eaten:
                u_state = State(curr.state.snake + (((curr_x - 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                u_state = State(curr.state.snake[1 : ] + (((curr_x - 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if u_state not in explored:
                explored.add(u_state)
                N = Node(u_state, curr.eaten + eaten)
                N.set_p_a(curr, 'U')
                queue.append(N)
            all_states += 1

        if ((((curr_x + 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x + 1) % n , curr_y % m) not in curr.state.snake:
            if eaten:
                d_state = State(curr.state.snake + (((curr_x + 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                d_state = State(curr.state.snake[1 : ] + (((curr_x + 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if d_state not in explored:
                explored.add(d_state)
                N = Node(d_state, curr.eaten + eaten)
                N.set_p_a(curr, 'D')
                queue.append(N)
            all_states += 1
            

        if (((curr_x % n, (curr_y - 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y - 1) % m)) not in curr.state.snake:
            if eaten:
                l_state = State(curr.state.snake + ((curr_x % n , (curr_y - 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                l_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y - 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if l_state not in explored:
                explored.add(l_state)
                N = Node(l_state, curr.eaten + eaten)
                N.set_p_a(curr, 'L')
                queue.append(N)
            all_states += 1
            

        if (((curr_x % n, (curr_y + 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y + 1) % m)) not in curr.state.snake:
            if eaten:
                r_state = State(curr.state.snake + ((curr_x % n , (curr_y + 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                r_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y + 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if r_state not in explored:
                explored.add(r_state)
                N = Node(r_state, curr.eaten + eaten)
                N.set_p_a(curr, 'R')
                queue.append(N)
            all_states += 1
                            

<strong>for our BFS function we use a queue to keep  our nodes. we check if it has a seed in it and then if it had  the snake will eat it. for our actions we have 4 conditions to go to right(<u> 'R'</u>), left(<u>'L'</u>), up(<u>'U'</u>) and down(<u>'D'</u>).<br>
for each actions we have similar conditions.<br>
for example we check the action to go right.<br>
    1. if we move to right, the snake wont bite its self( in other words : if <u>right block</u> not in curr.state.snake)<br>
    or<br>
    2. if the right block is snake's tail and the snake did not it any seeds in this state and length of snake is less than 2
    <br>
if each one of them is True then we can proceed to the right block.<br>
we know that BFS is complete and optimal if each move costs us equally, and in this case, moving to each nearby block only costs us 1, so BFS is optimal in our case.<br>
initially our state is where we start our game so state.snake = [ (curr_block) ] and state.seeds = seeds.<br>
for each state we should check 4 nearby blocks and append their state if they were not seen which we check with a set named <u> explored </u>. our action is the way we move and our final state or goal is where we have eaten all the seeds and none of them is available.</strong>


In [3]:
import time
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((hx, hy),)
    start = State(t, seeds)
    explored.add(start)
    queue = [Node(start, 0)]
    node, expand, all_states = BFS(queue, explored, n, m, num, seeds2)
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test1...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states :", expand)
print("all states :", all_states)
print("average time for 2 BFS runs : ", (t2 - t1) / 2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((hx, hy),)
    start = State(t, seeds)
    explored.add(start)
    queue = [Node(start, 0)]
    node, expand, all_states = BFS(queue, explored, n, m, num, seeds2)
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test2...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states :", expand)
print("all states :", all_states)
print("average time for 2 BFS runs : ", (t2 - t1) / 2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((hx, hy),)
    start = State(t, seeds)
    explored.add(start)
    queue = [Node(start, 0)]
    node, expand, all_states = BFS(queue, explored, n, m, num, seeds2)
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test3...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states :", expand)
print("all states :", all_states)
print("average time for 2 BFS runs : ", (t2 - t1) / 2, "seconds")



test1...
Path :
D-> L-> U-> U-> L-> U-> L-> L-> U-> U-> L-> L
12  moves
distinct states : 4571
all states : 13640
average time for 2 BFS runs :  0.06484901905059814 seconds
--------------------------------------------------------------------------------------------
test2...
Path :
U-> R-> D-> L-> L-> U-> U-> U-> U-> L-> U-> L-> L-> L-> L
15  moves
distinct states : 49004
all states : 144581
average time for 2 BFS runs :  0.7437463998794556 seconds
--------------------------------------------------------------------------------------------
test3...
Path :
U-> R-> D-> D-> D-> R-> D-> R-> R-> D-> D-> R-> R-> R-> U-> R-> R-> D-> L-> L-> L-> U-> U-> L-> L
25  moves
distinct states : 194831
all states : 563960
average time for 2 BFS runs :  4.547839641571045 seconds


<strong>

<strong> as we can see for evey bigger test, the number of states that we see are increasing and it is much more than the number of states that we expand</strong>

# IDS


In [4]:
#IDS function
def IDS(stack, n, m, num_of_seeds, seeds, deepening, dic):
    all_states = dist = 0
    while(stack):
        curr = stack[-1]
        dist += 1
        stack = stack[ : -1]
        if curr.depth > deepening:
            continue
        curr_x = curr.state.snake[-1][0]
        curr_y = curr.state.snake[-1][1]
        eaten = 0
        p = 0
        if ((curr_x, curr_y)) in seeds:
            idx = seeds.index((curr_x, curr_y))
            p = curr.state.seeds[idx][2]
            if p > 0:
                eaten = 1
                curr.taken = [curr_x, curr_y]
                idx = curr.state.seeds.index((curr_x, curr_y, p))
                if curr.eaten + 1 == num_of_seeds:
                    return curr, dist, all_states
        if ((((curr_x - 1) % n , curr_y % m)) in curr.state.snake and (((curr_x - 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or (((curr_x - 1) % n , curr_y % m)) not in curr.state.snake:
            if eaten:
                u_state = State(curr.state.snake + (((curr_x - 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                u_state = State(curr.state.snake[1 : ] + (((curr_x - 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if u_state not in dic or curr.depth + 1 < dic[u_state]:
                N = Node(u_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'U')
                stack.append(N)
                dic[u_state] = curr.depth + 1
            all_states += 1

        if ((((curr_x + 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x + 1) % n , curr_y % m) not in curr.state.snake:
            if eaten:
                d_state = State(curr.state.snake + (((curr_x + 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                d_state = State(curr.state.snake[1 : ] + (((curr_x + 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if d_state not in dic or curr.depth + 1 < dic[d_state]:
                N = Node(d_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'D')
                stack.append(N)
                dic[d_state] = curr.depth + 1
            all_states += 1
            
        if (((curr_x % n, (curr_y - 1) % m)) in curr.state.snake and ((curr_x % n, (curr_y - 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y - 1) % m)) not in curr.state.snake:
            if eaten:
                l_state = State(curr.state.snake + ((curr_x % n , (curr_y - 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                l_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y - 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if l_state not in dic or curr.depth + 1 < dic[l_state]:
                N = Node(l_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'L')
                stack.append(N)
                dic[l_state] = curr.depth + 1
            all_states += 1
            
        if (((curr_x % n, (curr_y + 1) % m)) in curr.state.snake and ((curr_x % n, (curr_y + 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y + 1) % m)) not in curr.state.snake:
            if eaten:
                r_state = State(curr.state.snake + ((curr_x % n , (curr_y + 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                r_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y + 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if r_state not in dic or curr.depth + 1 < dic[r_state]:
                N = Node(r_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'R')
                stack.append(N)
                dic[r_state] = curr.depth + 1
            all_states += 1
            
    return None, 0, 0

<strong> when we implement IDS function we need an upper limit for the depth we are going to reach. and also we should use a stack instead of a queue to implement the DFS. when we reach the limit in a node and we haven't reached our gola state, we should skip expanding this node and not go any further than this state. but the conditions to expand each state is the same as BFS we implemented.our initial state is just like before, but the goal state here has 2 conditions.<br>
    1. if you have eaten all the seeds, then return this node as goal.<br>
    2. if you have reached the limit and haven't found any final states and the stack is empty, return failure and increase the upper limit.<br>
just like the bfs, IDS is complete and because the costs between two adjacent states are equal, here we have an optimal IDS.<br>
but here we should check another condition. we may have seen a state with value C1 and we will see it again but with cost C2, if C2 is less than C1 it means that we can see a state which we have seen before but we see it with less cost so we should add it to the stack again. we check this condition using a dictionary instead of a list to check the explored.
</strong>

In [5]:
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    stack = [Node(start, 0, 0)]
    max_depth = 10 ** 9
    for i in range(max_depth):
        dic = dict()
        start = State(t, seeds)
        stack = [Node(start, 0, 0)]
        dic[start] = 0
        node, dist, all_states = IDS(stack, n, m, num, seeds2, i, dic)
        if node:
            break
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test1...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states: ", dist)
print("states: ", all_states)
print("average time for 2 IDS runs : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    stack = [Node(start, 0, 0)]
    max_depth = 10 ** 9
    for i in range(max_depth):
        dic = dict()
        start = State(t, seeds)
        stack = [Node(start, 0, 0)]
        dic[start] = 0
        node, dist, all_states = IDS(stack, n, m, num, seeds2, i, dic)
        if node:
            break
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test2...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states: ", dist)
print("all states: ", all_states)
print("average time for 2 IDS runs : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    stack = [Node(start, 0, 0)]
    max_depth = 10 ** 9
    for i in range(max_depth):
        dic = dict()
        start = State(t, seeds)
        stack = [Node(start, 0, 0)]
        dic[start] = 0
        node, dist, all_states = IDS(stack, n, m, num, seeds2, i, dic)
        if node:
            break
t2 = time.time()
path = []
while(node.parent):
    path.append(node.action)
    node = node.parent
length = len(path)
string = ""
num_of_moves = 1
for i in range(length - 1, 0 , -1):
    string += path[i] + "-> "
    num_of_moves += 1
string += path[0]
print("test3...")
print("Path :")
print(string)
print(num_of_moves, " moves")
print("distinct states: ", dist)
print("all states: ", all_states)
print("average time for 2 IDS runs : ", (t2 - t1)/2, "seconds")


test1...
Path :
L-> D-> R-> R-> D-> D-> R-> R-> D-> R-> D-> D
12  moves
distinct states:  13740
states:  27473
average time for 2 IDS runs :  0.4761340618133545 seconds
--------------------------------------------------------------------------------------------
test2...
Path :
R-> L-> L-> U-> R-> U-> L-> L-> U-> U-> L-> L-> L-> L-> U
15  moves
distinct states:  50399
all states:  99466
average time for 2 IDS runs :  3.6663025617599487 seconds
--------------------------------------------------------------------------------------------
test3...
Path :
R-> U-> R-> D-> D-> D-> R-> R-> D-> R-> R-> R-> D-> D-> R-> R-> U-> L-> L-> D-> L-> L-> L-> U-> U
25  moves
distinct states:  522147
all states:  1145675
average time for 2 IDS runs :  22.388766050338745 seconds


<strong> as you can see it is much slower than BFS or A* search which we will see further in the report.<br>
this much time is used because when we reach a limit we throw away every thing and start over for a new limit hich we can avoid and make it faster but it is not the point of this assignment.<br>
and also another point, expanded states and seen states in IDS is much more than BFS.</strong>
<strong> but it is not just about the time.<br>
<strong>if we go back to the BFS results we can see that for every test, the states that we see and the states that we expand are alot bigger in IDS.<br>
but just like before expanded states are much less than seen states.</strong>

# Weighted A*

In [6]:
#needed function for A* search
#admissible and consistent heuristic
def find_heuristic(node, num_of_seeds):
    return num_of_seeds - (node.eaten + 1)


<strong> find_heuristic is a function we are going to use for our admissibile and consistent heuristic.<br>
i considered this heuristic because for every state we eat a seed it makes it different than the others and we choose the heuristic to be <u> number of seeds left in the game </u>. when we reach a depth and lets's say we have found more seeds than the other states in the list, we can say that it is better than almost every other state in the list due to less seeds remaining and so probably less moves remaining to the goal.<br>
it is consistent because:<br>
1. for every step, if we eat a seed, the difference of the heuristics of the adjacent states will be 1, and if we don't it will be 0. then the difference will at most be 1.<br>
2. for every state we know that the cost to goal is $\geq$ 1.<br>
3. therefor we have for every state: realcost $\geq$ 1 and is always $\ge$ 0. so our heuristic is constistent.<br></strong>
<strong>
it is admissible because:<br>
    1.let's consider the most and the best way to finish the game.<br>we have k seeds and they are put next to each other in the table.<br>
    2.we know that the best way to end the game is to eat a seed at each step, therefor we finish the game in k steps.<br>we considered the best and most optimal way to finish a game is to finish it in k steps and we are sure that it can not be done in less than k steps(because we have k seeds as we said!).<br>
    3.so for every state, the remaining steps can not be less than the number of remaining seeds. therefor we have an admissible heuristic.<br></strong>


In [7]:
#just admissible heuristic
def find_heuristic2(node, num_of_seeds):
    return (num_of_seeds - (node.eaten + 1)) / node.depth

<strong> find_heuristic2 is another admissible heuristic.(it is not consistent)<br>
for every state if we move, the equation becomes smaller and if we eat a seed, the remaining seeds become smaller causing the equation become smaller so we can almost say that by becoming smaller we have eaten more seeds.
it is not consistent because:<br>
1.if we eat a seed in the for exmaple 4 first steps for each step depending on the number of seeds, heuristic(S2) - heuristic(S1) may be much more bigger than 1 which is the cost between two steps in our case. so it is not consistent.<br></strong>
<strong>
it is admissible because: just like the last heuristic.<br>
    1.let's consider the most and the best way to finish the game.<br>we have k seeds and they are put next to each other in the table.<br>
    2.we know that the best way to end the game is to eat a seed at each step, therefor we finish the game in k steps.<br>we considered the best and most optimal way to finish a game is to finish it in k steps and we are sure that it can not be done in less than k steps(because we have k seeds as we said!).<br>
    3.so for every state, the remaining steps can not be less than the number of remaining seeds. therefor we have an admissible heuristic.<br></strong>
<strong>and with every step it becomes smaller than before so it is never bigger than the real cost to the last state therefor it i admissible</strong>


In [8]:
def Weighted_Astar(queue, explored, n, m, num_of_seeds, seeds, weight = 1):
    exp = 0
    all_state = 0
    while(queue):
        curr = queue.del_min()
        exp += 1
        curr_x = curr.state.snake[-1][0]
        curr_y = curr.state.snake[-1][1]
        eaten = 0
        p = 0
        if ((curr_x, curr_y)) in seeds:
            idx = seeds.index((curr_x, curr_y))
            p = curr.state.seeds[idx][2]
            if p > 0:
                eaten = 1
                idx = curr.state.seeds.index((curr_x, curr_y, p))
                if curr.eaten + 1 == num_of_seeds:
                    return curr, exp, all_state

        if ((((curr_x - 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or (((curr_x - 1) % n , curr_y % m)) not in curr.state.snake:
            if eaten:
                u_state = State(curr.state.snake + (((curr_x - 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                u_state = State(curr.state.snake[1 : ] + (((curr_x - 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if u_state not in explored:
                explored.add(u_state)
                N = Node(u_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'U')
                N.heuristic = find_heuristic(N, num_of_seeds) * weight
                N.almost = N.depth + (N.heuristic)
                queue.add(N)
            all_state += 1

        if ((((curr_x + 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x + 1) % n , curr_y % m) not in curr.state.snake:
            if eaten:
                d_state = State(curr.state.snake + (((curr_x + 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                d_state = State(curr.state.snake[1 : ] + (((curr_x + 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if d_state not in explored:
                explored.add(d_state)
                N = Node(d_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'D')
                N.heuristic = find_heuristic(N, num_of_seeds) * weight
                N.almost = N.depth + (N.heuristic)
                queue.add(N)
            all_state += 1
            

        if (((curr_x % n, (curr_y - 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y - 1) % m)) not in curr.state.snake:
            if eaten:
                l_state = State(curr.state.snake + ((curr_x % n , (curr_y - 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                l_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y - 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if l_state not in explored:
                explored.add(l_state)
                N = Node(l_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'L')
                N.heuristic = find_heuristic(N, num_of_seeds) * weight
                N.almost = N.depth + (N.heuristic)
                queue.add(N)
            all_state += 1
            
        
        if (((curr_x % n, (curr_y + 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y + 1) % m)) not in curr.state.snake:
            if eaten:
                r_state = State(curr.state.snake + ((curr_x % n , (curr_y + 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                r_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y + 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if r_state not in explored:
                explored.add(r_state)
                N = Node(r_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'R')
                N.heuristic = find_heuristic(N, num_of_seeds) * weight
                N.almost = N.depth + (N.heuristic)
                queue.add(N)
            all_state += 1
            

<strong> to implement weighted A* search function we need a function to find heuristic which in this case it is find_heuristic() that we discussed it above. we also need a weight which is <u>1</u> by default.for every node we have a field called <u>almost</u>. almost is equal to heuristic plus cost to this state which is depth. we then sort the list depending on the almost value of each node. (of course we use a min heap here).<br>
initial state, goal state and action is just like before in BFS.<br>
we know that for a consistent heuristic it is admissible and also optimal.
here we used a consistent heuristic therefor we expect an optimal solution which we will check below.<br>
and it does not depend on the cost of transition between each node and is optimal for every consistent heuristic.</strong>


In [9]:
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test1... ")
print("Path for weight 1 : ",string)
print("moves for weight 1:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1 : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 1.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test1... ")
print("Path for weight 1.75 : ",string)
print("moves for weight 1.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1.75 : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 4.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test1... ")
print("Path for weight 4.75 : ",string)
print("moves for weight 4.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 4.75 : ", (t2 - t1)/2, "seconds")


test1... 
Path for weight 1 :  D -> L -> L -> U -> U -> U -> L -> L -> U -> L -> L -> U
moves for weight 1: 12
distinct states : 2685
all states : 8028
average time for weighted A* weight = 1 :  0.14991235733032227 seconds
--------------------------------------------------------------------------------------------
test1... 
Path for weight 1.75 :  D -> L -> U -> U -> L -> U -> L -> L -> L -> U -> L -> U
moves for weight 1.75: 12
distinct states : 1196
all states : 3580
average time for weighted A* weight = 1.75 :  0.021085739135742188 seconds
--------------------------------------------------------------------------------------------
test1... 
Path for weight 4.75 :  D -> L -> U -> U -> L -> U -> L -> L -> L -> U -> U -> L
moves for weight 4.75: 12
distinct states : 95
all states : 278
average time for weighted A* weight = 4.75 :  0.0022296905517578125 seconds


In [10]:
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test2... ")
print("Path for weight 1 : ",string)
print("moves for weight 1:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1 : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 1.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test2... ")
print("Path for weight 1.75 : ",string)
print("moves for weight 1.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1.75 : ", (t2 - t1)/2, "seconds")
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 4.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("--------------------------------------------------------------------------------------------")
print("test2... ")
print("Path for weight 4.75 : ",string)
print("moves for weight 4.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 4.75 : ", (t2 - t1)/2, "seconds")


test2... 
Path for weight 1 :  R -> L -> L -> U -> R -> U -> U -> U -> L -> L -> U -> L -> L -> L -> L
moves for weight 1: 15
distinct states : 26217
all states : 77340
average time for weighted A* weight = 1 :  0.648425817489624 seconds
--------------------------------------------------------------------------------------------
test2... 
Path for weight 1.75 :  R -> L -> L -> U -> R -> U -> U -> L -> U -> L -> L -> L -> U -> L -> L
moves for weight 1.75: 15
distinct states : 17770
all states : 52014
average time for weighted A* weight = 1.75 :  0.40840959548950195 seconds
--------------------------------------------------------------------------------------------
test2... 
Path for weight 4.75 :  U -> L -> D -> R -> R -> U -> L -> L -> U -> U -> U -> L -> U -> L -> L -> L -> L
moves for weight 4.75: 17
distinct states : 2315
all states : 6573
average time for weighted A* weight = 4.75 :  0.049541592597961426 seconds


In [11]:
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test3... ")
print("Path for weight 1 : ",string)
print("moves for weight 1:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1 : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 1.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test3... ")
print("Path for weight 1.75 : ",string)
print("moves for weight 1.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 1.75 : ", (t2 - t1)/2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node1, exp1, all_states = Weighted_Astar(heap, explored, n, m, num, seeds2, 4.75)
t2 = time.time()
str = []
while(node1.parent):
    str.append(node1.action)
    node1 = node1.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test3... ")
print("Path for weight 4.75 : ",string)
print("moves for weight 4.75:", length)
print("distinct states :", exp1)
print("all states :", all_states)
print("average time for weighted A* weight = 4.75 : ", (t2 - t1)/2, "seconds")


test3... 
Path for weight 1 :  U -> R -> D -> D -> R -> D -> R -> R -> D -> R -> R -> D -> R -> D -> R -> R -> U -> L -> L -> D -> L -> L -> L -> U -> U
moves for weight 1: 25
distinct states : 101584
all states : 296524
average time for weighted A* weight = 1 :  2.5637900829315186 seconds
--------------------------------------------------------------------------------------------
test3... 
Path for weight 1.75 :  U -> R -> D -> D -> R -> D -> D -> R -> R -> D -> R -> D -> R -> R -> R -> R -> U -> L -> L -> D -> L -> L -> U -> L -> U
moves for weight 1.75: 25
distinct states : 57793
all states : 168630
average time for weighted A* weight = 1.75 :  1.7104480266571045 seconds
--------------------------------------------------------------------------------------------
test3... 
Path for weight 4.75 :  U -> R -> D -> D -> D -> R -> D -> R -> R -> U -> L -> D -> R -> R -> D -> R -> D -> R -> R -> R -> U -> U -> L -> D -> L -> D
moves for weight 4.75: 26
distinct states : 909
all states : 26

<strong> we can see in the weighted A* search for all the tests we have a faster search than IDS and BFS.much faster !<br>
for every test we have used an $\alpha$ as weight for our function.<br>
for test 1 we can see that for $\alpha$ = 1 and $\alpha$ = 1.75 and $\alpha$ = 4.75 we have optimal solution for each one of them.<br>
for test 2 we have an optimal solution for $\alpha$ = 1 and $\alpha$ = 1.75 but for $\alpha$ = 4.75 we have 17 moves which is not optimal and we knew that would happen.<br>
fr every $\alpha$ $\neq$ 1 the solution may be suboptimal and be at most $\alpha$ times worse than optimal solution.<br>
this happened for test 3 too. for $\alpha$ = 1 and $\alpha$ = 1.75 we havve optimal solution but for $\alpha$ = 4.75 it is not optimal.<br>
and A* as we can see is much more powerful and faster than BFS or IDS and also look at the number of expanded and seen states. it is so much less than the other two search algorithms. and when we use a weight $\neq$ 1 the bigger the $\alpha$ gets the smaller the number of expanded states and seen states get.</strong>
<strong> this happens because when we multiply it by almost 2, estimation becomes much more like the real cost, but when we multiply it by almost 5, it grows more than the real cost and becomes worse so that it gives us suboptimal solution</strong>

# A*

In [12]:
# we use A* function with weight 1
def Astar(heap, explored, n, m, num_of_seeds, seeds):
    exp = 1
    all_states = 1
    while(heap):
        curr = heap.del_min()
        exp += 1
        curr_x = curr.state.snake[-1][0]
        curr_y = curr.state.snake[-1][1]
        eaten = 0
        p = 0
        if ((curr_x, curr_y)) in seeds:
            idx = seeds.index((curr_x, curr_y))
            p = curr.state.seeds[idx][2]
            if p > 0:
                eaten = 1
                idx = curr.state.seeds.index((curr_x, curr_y, p))
                if curr.eaten + 1 == num_of_seeds:
                    return curr, exp, all_states

        if ((((curr_x - 1) % n , curr_y % m)) in curr.state.snake and (((curr_x - 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or (((curr_x - 1) % n , curr_y % m)) not in curr.state.snake:
            if eaten:
                u_state = State(curr.state.snake + (((curr_x - 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                u_state = State(curr.state.snake[1 : ] + (((curr_x - 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if u_state not in explored:
                explored.add(u_state)
                N = Node(u_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'U')
                N.heuristic = find_heuristic2(N, num_of_seeds)
                N.almost = N.depth + (N.heuristic)
                heap.add(N)
            all_states += 1

        if ((((curr_x + 1) % n , curr_y % m)) in curr.state.snake and (((curr_x + 1) % n , curr_y % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x + 1) % n , curr_y % m) not in curr.state.snake:
            if eaten:
                d_state = State(curr.state.snake + (((curr_x + 1) % n , curr_y),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                d_state = State(curr.state.snake[1 : ] + (((curr_x + 1) % n , curr_y),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if d_state not in explored:
                explored.add(d_state)
                N = Node(d_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'D')
                N.heuristic = find_heuristic2(N, num_of_seeds)
                N.almost = N.depth + (N.heuristic)
                heap.add(N)
            all_states += 1

        if (((curr_x % n, (curr_y - 1) % m)) in curr.state.snake and ((curr_x % n, (curr_y - 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y - 1) % m)) not in curr.state.snake:
            if eaten:
                l_state = State(curr.state.snake + ((curr_x % n , (curr_y - 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                l_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y - 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if l_state not in explored:
                explored.add(l_state)
                N = Node(l_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'L')
                N.heuristic = find_heuristic2(N, num_of_seeds)
                N.almost = N.depth + (N.heuristic)
                heap.add(N)
            all_states += 1

        if (((curr_x % n, (curr_y + 1) % m)) in curr.state.snake and ((curr_x % n, (curr_y + 1) % m)) == curr.state.snake[0] and curr.length > 2 and eaten != 1) or ((curr_x % n, (curr_y + 1) % m)) not in curr.state.snake:
            if eaten:
                r_state = State(curr.state.snake + ((curr_x % n , (curr_y + 1) % m),), curr.state.seeds[0 : idx] + ((curr_x, curr_y, p - 1), ) + curr.state.seeds[idx + 1 : ])
            else:
                r_state = State(curr.state.snake[1 : ] + ((curr_x  % n , (curr_y + 1) % m),), curr.state.seeds[ : -1] + ((curr.state.seeds[-1][0], curr.state.seeds[-1][1], curr.state.seeds[-1][2]),))
            if r_state not in explored:
                explored.add(r_state)
                N = Node(r_state, curr.eaten + eaten, curr.depth + 1)
                N.set_p_a(curr, 'R')
                N.heuristic = find_heuristic2(N, num_of_seeds)
                N.almost = N.depth + (N.heuristic)
                heap.add(N)
            all_states += 1

<strong> the heuristic we used for this function was not consistent and we know that it might not give an optimal solution. we will see the results below.</strong>

In [13]:
t1 = time.time()
for i in range(2):
    f = open("test1.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node, exp, all_states = Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node.parent):
    str.append(node.action)
    node = node.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test1...")
print("Path : ",string)
print("moves:", length)
print("distinct states :", exp)
print("all states :", all_states)
print("average time for 2 A* runs : ", (t2 - t1) / 2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test2.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node, exp, all_states = Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node.parent):
    str.append(node.action)
    node = node.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test2...")
print("Path : ",string)
print("moves:", length)
print("distinct states :", exp)
print("all states :", all_states)
print("average time for 2 A* runs : ", (t2 - t1) / 2, "seconds")
print("--------------------------------------------------------------------------------------------")
t1 = time.time()
for i in range(2):
    f = open("test3.txt", "r")
    n, m = [int(x) for x in f.readline().split(',')]
    hx, hy = [int(x) for x in f.readline().split(',')]
    num_of_seeds = int(f.readline())
    seeds = ()
    seeds2 = ()
    num = 0
    for i in range(num_of_seeds):
         x,y,s = [int(j) for j in f.readline().split(',')]
         seeds += ((x, y, s), )
         seeds2 += ((x, y), )
         num += s
    explored = set()
    t = ()
    t += ((int(hx), int(hy)),)
    start = State(t, seeds)
    explored.add(start)
    heap = Heap()
    heap.add(Node(start, 0, 0))
    node, exp, all_states = Astar(heap, explored, n, m, num, seeds2)
t2 = time.time()
str = []
while(node.parent):
    str.append(node.action)
    node = node.parent
length = len(str)
string = ""
for i in range(length - 1, 0 , -1):
    string += str[i] + " -> "
string += str[0]
print("test3...")
print("Path : ",string)
print("moves:", length)
print("distinct states :", exp)
print("all states :", all_states)
print("average time for 2 A* runs : ", (t2 - t1) / 2, "seconds")

test1...
Path :  L -> D -> L -> D -> R -> U -> U -> U -> L -> U -> L -> L
moves: 12
distinct states : 3814
all states : 11418
average time for 2 A* runs :  0.06936049461364746 seconds
--------------------------------------------------------------------------------------------
test2...
Path :  R -> L -> L -> U -> R -> U -> U -> U -> L -> L -> U -> L -> L -> L -> L
moves: 15
distinct states : 40533
all states : 119929
average time for 2 A* runs :  0.9248683452606201 seconds
--------------------------------------------------------------------------------------------
test3...
Path :  U -> R -> D -> D -> R -> D -> D -> R -> R -> D -> R -> D -> R -> R -> R -> R -> U -> L -> L -> D -> L -> U -> L -> L -> U
moves: 25
distinct states : 162997
all states : 473340
average time for 2 A* runs :  4.466643452644348 seconds


<strong>as we can see it is still very much faster thann both BFS and IDS  and it is optimal too, but about the number of states seen,it is almost just like BFS, only a little less, but differs from IDS alot and it is much less in A* with an admissible heuristic and it is not as good as the weighted A* which we used a consistent heuristic.</strong>

# Test 1 Results

|       |path cost|            path             |all states|distinct states|average time|
|:-----:|:-------:|:---------------------------:|:--------:|:-------------:|:----------:|
|BFS    |12|D,L,U,U,L,U,L,L,U,U,L,L|13640|4571|0.0648|
|IDS    |12|L,D,R,R,D,D,R,R,D,R,D,D|27473|13740|0.4761|
|weighted A*,$\alpha$ = 1|12|D,L,L,U,U,U,L,L,U,L,L,U|8028|2685|0.1499|
|weighted A*,$\alpha$ = 1.75|12|D,L,U,U,L,U,L,L,L,U,L,U|3580|1196|0.0210|
|weighted A*,$\alpha$ = 4.75|12|D,L,U,U,L,U,L,L,L,U,U,L|278|95|0.0022|
|normal A*|12|L,D,L,D,R,U,U,U,L,U,L,L|11418|3814|0.0693|


# Test 2 Results

|       |path cost|            path             |all states|distinct states|average time|
|:-----:|:-------:|:---------------------------:|:--------:|:-------------:|:----------:|
|BFS    |15|U,R,D,L,L,U,U,U,U,L,U,L,L,L,L|144581|49004|0.7437|
|IDS    |15|R,L,L,U,R,U,L,L,U,U,L,L,L,L,U|99466|50399|3.666|
|weighted A*,$\alpha$ = 1|15|R,L,L,U,R,U,U,L,U,L,L,L,U,L,L|77340|26217|0.6484|
|weighted A*,$\alpha$ = 1.75|15|R,L,L,U,R,U,U,L,U,L,L,L,U,L,L|52014|17770|0.4084|
|weighted A*,$\alpha$ = 4.75|17|U,L,D,R,R,U,L,L,U,U,U,L,U,L,L,L,L|6573|2315|0.0495|
|normal A*|15|R,L,L,U,R,U,U,U,L,L,U,L,L,L,L|119929|40533|0.9248|

# Test 3 Results

|       |path cost|            path             |all states|distinct states|average time|
|:-----:|:-------:|:---------------------------:|:--------:|:-------------:|:----------:|
|BFS    |25|U,R,D,D,D,R,D,R,R,D,D,R,R,R,U,R,R,D,L,L,L,U,U,L,L|563960|194831|4.5478|
|IDS    |25|R,U,R,D,D,D,R,R,D,R,R,R,D,D,R,R,U,L,L,D,L,L,L,U,U|1145675|522147|22.3887|
|weighted A*,$\alpha$ = 1|25|U,R,D,D,R,D,R,R,D,R,R,D,R,D,R,R,U,L,L,D,L,L,L,U,U|296524|101584|2.5637|
|weighted A*,$\alpha$ = 1.75|25|U,R,D,D,R,D,D,R,R,D,R,D,R,R,R,R,U,L,L,D,L,L,U,L,U,|168630|57793|1.7104|
|weighted A*,$\alpha$ = 4.75|26|U,R,D,D,D,R,D,R,R,U,L,D,R,R,D,R,D,R,R,R,U,U,L,D,L,D|2601|909|0.0559|
|normal A*|25|U,R,D,D,R,D,D,R,R,D,R,D,R,R,R,R,U,L,L,D,L,U,L,L,U|473341|162997|4.4666|


<strong>so we conclude that for search algorithms, the fastest one is an A* search with consistent heuristic, then A* with just admissible heuristic, BFS and then IDS. A* with consistent heuristic, not only has the fastest calculations, but also needs less memory which leads us to believe that it is the best search algorithm among these 4 types that we checked.</strong>